## <span style="color:teal"> __Data Abundance table__

In [1]:
import pandas as pd
import json

In [2]:
# === Step 1: Load the data ===
df = pd.read_csv(
    "/mnt/iusers01/fatpou01/bmh01/msc-bioinf-2024-2025/h44063jg/gm_repository/species_abundance.txt",
    sep="\t"
)

# === Step 2: Define ID columns used to identify duplicates ===
id_cols = ['loaded_uid', 'ncbi_taxon_id', 'taxon_rank_level']

# === Step 3: Identify abundance columns (assumed numeric, exclude ID columns) ===
abundance_cols = df.columns.difference(id_cols)

# === Step 4: Check for duplicated rows ===
duplicate_rows = df[df.duplicated(subset=id_cols, keep=False)]

# === Step 5: Summary before deduplication ===
print("=== Summary Before Deduplication ===")
print(f"Total rows in original DataFrame: {len(df)}")
print(f"Number of duplicated rows: {len(duplicate_rows)}")
print(f"Number of unique groups (by {id_cols}): {df[id_cols].drop_duplicates().shape[0]}")
print()

# === Step 6: Deduplicate by averaging relative abundance values ===
df = df.groupby(id_cols, as_index=False)[abundance_cols].mean()

# === Step 7: Summary after deduplication ===
print("=== Summary After Deduplication ===")
print(f"Total rows after deduplication: {len(df)}")
print(f"Shape of final DataFrame: {df.shape}")
print("\nDeduplication complete. Duplicates collapsed by averaging abundance values.\n")

# === Step 8: Taxonomic Rank Summary ===
print("=== Taxonomic Rank Distribution ===")
rank_counts = df['taxon_rank_level'].value_counts()
print(rank_counts)

# Focused summary for genus and species
n_genus = rank_counts.get('genus', 0)
n_species = rank_counts.get('species', 0)

print(f"\nNumber of Genus-Level entries: {n_genus}")
print(f"Number of Species-Level entries: {n_species}")

# === Step 9: Rows with ncbi_taxon_id == -1 ===
n_unclassified = df[df['ncbi_taxon_id'] == -1].shape[0]
print(f"\nNumber of rows with ncbi_taxon_id = -1 (unclassified): {n_unclassified}")


=== Summary Before Deduplication ===
Total rows in original DataFrame: 8283371
Number of duplicated rows: 574
Number of unique groups (by ['loaded_uid', 'ncbi_taxon_id', 'taxon_rank_level']): 8283084

=== Summary After Deduplication ===
Total rows after deduplication: 8283084
Shape of final DataFrame: (8283084, 4)

Deduplication complete. Duplicates collapsed by averaging abundance values.

=== Taxonomic Rank Distribution ===
taxon_rank_level
species    5751183
genus      2531901
Name: count, dtype: int64

Number of Genus-Level entries: 2531901
Number of Species-Level entries: 5751183

Number of rows with ncbi_taxon_id = -1 (unclassified): 64488


In [3]:
duplicate_rows = df[df.duplicated(subset=id_cols, keep=False)]
duplicate_rows

,loaded_uid,ncbi_taxon_id,taxon_rank_level,relative_abundance


___

In [4]:
# DIAGNOSTICS ON MAPPING TABLES
# === Paths ===
PATH_DF2 = "/mnt/iusers01/fatpou01/bmh01/msc-bioinf-2024-2025/h44063jg/gm_repository/samples_loaded.txt"
PATH_DF3 = "/mnt/iusers01/fatpou01/bmh01/msc-bioinf-2024-2025/h44063jg/gm_repository/sample_to_disease_info.txt"

# === Load mapping tables ===
df2 = pd.read_csv(PATH_DF2, sep="\t")  # expects: uid, accession_id
df3 = pd.read_csv(PATH_DF3, sep="\t")  # expects: run_id, disease

# ---------- Compute (df2) ----------
req2 = {"uid", "accession_id"}
miss2 = req2 - set(df2.columns)
if miss2: raise KeyError(f"df2 missing columns: {miss2}")

df2_shape = df2.shape
df2_nulls = df2[["uid","accession_id"]].isna().sum().to_dict()
df2_pair_dup_rows = int(df2.duplicated(["uid","accession_id"], keep=False).sum())
df2_uid_conflicts = int((df2.groupby("uid")["accession_id"].nunique() > 1).sum())
df2_acc_conflicts = int((df2.groupby("accession_id")["uid"].nunique() > 1).sum())

# ---------- RESULTS ----------
print("\n" + "="*100)
print("RESULTS — df2 Sample Loading Info (uid → accession_id)")
print("-"*100)
print(f"Shape: {df2_shape}")
print(f"Nulls (uid/accession_id): {df2_nulls}")
print(f"Exact (uid, accession_id) duplicate rows: {df2_pair_dup_rows}")
print(f"uids mapping to >1 accession_id: {df2_uid_conflicts}")
print(f"accession_id mapping to >1 uid: {df2_acc_conflicts}")

# ---------- Compute (df3) ----------
# 1) Unique (run_id, disease) pairs (exclude no-disease)
pairs = (df3[['run_id', 'disease']]
         .dropna(subset=['disease'])
         .drop_duplicates())

print("Unique (run_id, disease) pairs:", len(pairs))

# 2) Which pairs were duplicated in the raw data (optional)
dup_pairs = (df3[['run_id', 'disease']]
             .dropna(subset=['disease'])
             .value_counts()
             .reset_index(name='count')
             .query('count > 1')
             .sort_values('count', ascending=False))
print("\n" + "="*100)
print("RESULTS — df3 Sample to disease Info (run_id → disease)")
print("-"*100)
print("Duplicated pairs (count > 1):", len(dup_pairs))

# Assumes `pairs` exists and has columns: run_id, disease
# (unique pairs with no missing disease)

# Count diseases per run_id
counts = pairs.groupby('run_id')['disease'].nunique().rename('n_diseases')

# Identify groups
multi_run_ids  = counts[counts > 1].index            # comorbid run_ids
single_run_ids = counts[counts == 1].index           # single-disease run_ids

# Exploded rows for comorbid run_ids (ready for merging)
pairs_multi  = pairs[pairs['run_id'].isin(multi_run_ids)].copy()

# One row per single-disease run_id (keep the only disease)
pairs_single = (pairs[pairs['run_id'].isin(single_run_ids)]
                .drop_duplicates('run_id')
                .copy())

# Optional: readable summary of comorbids
multi_summary = (
    pairs_multi.groupby('run_id')
               .agg(n_diseases=('disease','nunique'),
                    diseases=('disease', lambda s: ', '.join(sorted(s.unique()))))
               .sort_values('n_diseases', ascending=False)
)

print(f"Total unique run_ids in pairs (after removing the pairs - deduplicated): {counts.shape[0]}")
print(f"Single-disease run_ids:        {len(single_run_ids)}")
print(f"Multi-disease (comorbid):      {len(multi_run_ids)}")


# Assumes `pairs` has columns: run_id, disease (unique pairs; no NaNs)

# Count unique diseases per run_id
n_dis = pairs.groupby('run_id')['disease'].nunique()

# Focus on multi-disease only
multi_mask = n_dis > 1
multi_counts = n_dis[multi_mask]

# Totals
n_multi_total = int(multi_counts.shape[0])
n_exact2      = int((multi_counts == 2).sum())
n_gt2         = int((multi_counts > 2).sum())

# Full breakdown (e.g., 2→#, 3→#, 4→#, ...)
breakdown = multi_counts.value_counts().sort_index()

print("\n" + "="*100)
print("=== Multi-disease run_ids breakdown ===")
print(f"Total multi-disease (run_ids >1 disease): {n_multi_total}")
print(f"  exactly 2 diseases:                     {n_exact2}")
print(f"  >2 diseases:                            {n_gt2}")
# Summary: counts by number of diseases
print("\n" + "="*100)
print("=== Counts by number of diseases per run_id ===")
print(n_dis.value_counts().sort_index())
print("="*100)

# pairs: columns ['run_id', 'disease'], unique pairs, no NaNs
# Ensure strings (optional but safe)
pairs_use = pairs.assign(
    run_id=pairs['run_id'].astype(str),
    disease=pairs['disease'].astype(str)
)

# Dict: run_id -> [disease1, disease2, ...]
run_to_diseases = (
    pairs_use.groupby('run_id')['disease']
             .apply(lambda s: sorted(set(s)))
             .to_dict()
)
# save as json dictionary
with open("run_to_diseases.json", "w") as f:
    json.dump(run_to_diseases, f, indent=2)

# Merge accession_id into df
df = df.merge(
    df2[['uid', 'accession_id']],
    how='left',
    left_on='loaded_uid',
    right_on='uid'  # df2 has unique uid
)

# Drop helper column from df2
df.drop(columns='uid', inplace=True)


# Assign disease list to df (None/NaN where no mapping)
run_to_diseases = {str(k).strip(): v for k, v in run_to_diseases.items()}
df['disease_list'] = df['accession_id'].map(run_to_diseases)

# Merge, Drop empty rows, explode, rename disease_list to disease and reset index
df = (df.dropna(subset=['disease_list'])
        .explode('disease_list')
        .rename(columns={'disease_list':'disease'})
        .drop(columns='accession_id') 
        .reset_index(drop=True))

# split into df_genus and df_species
df_genus   = df[df['taxon_rank_level'].astype(str).str.strip().str.lower().eq('genus')].copy()
df_species = df[df['taxon_rank_level'].astype(str).str.strip().str.lower().eq('species')].copy()

# save to csv
df_genus.to_csv("abundance_genus_data.txt", sep="\t", index=False)
df_species.to_csv("abundance_species_data.txt", sep="\t", index=False)





RESULTS — df2 Sample Loading Info (uid → accession_id)
----------------------------------------------------------------------------------------------------
Shape: (52633, 8)
Nulls (uid/accession_id): {'uid': 0, 'accession_id': 0}
Exact (uid, accession_id) duplicate rows: 0
uids mapping to >1 accession_id: 0
accession_id mapping to >1 uid: 0
Unique (run_id, disease) pairs: 59051

RESULTS — df3 Sample to disease Info (run_id → disease)
----------------------------------------------------------------------------------------------------
Duplicated pairs (count > 1): 98
Total unique run_ids in pairs (after removing the pairs - deduplicated): 51934
Single-disease run_ids:        48349
Multi-disease (comorbid):      3585

=== Multi-disease run_ids breakdown ===
Total multi-disease (run_ids >1 disease): 3585
  exactly 2 diseases:                     1857
  >2 diseases:                            1728

=== Counts by number of diseases per run_id ===
disease
1     48349
2      1857
3       826
